In [77]:
from scipy.optimize import linprog
import re
import numpy as np


with open('input.txt', 'r') as f:
    lines = f.readlines()

line_0_1_reg = re.compile(r'Button [AB]: X\+(\d+), Y\+(\d+)')
line_2_reg = re.compile(r'Prize: X=(\d+), Y=(\d+)')

problems = []
current_problem = []
for i, line in enumerate(lines):
    line_type = i % 4
    if line_type == 3:
        problems.append(current_problem)
        current_problem = []
        continue
    if line_type == 2:
        current_problem.append(line_2_reg.match(line).groups())
    else:
        current_problem.append(line_0_1_reg.match(line).groups())
problems.append(current_problem)       
problems = np.array(problems, dtype=np.int64)

def return_optimal_solution(problem: np.ndarray):
    c = [3, 1]
    A = [problem[:2, 0], problem[:2, 1]]
    b = problem[2]
    a_bounds = (0, None)
    b_bounds = (0, None)
    bounds = [a_bounds, b_bounds]
    res = linprog(c, A_eq=A, b_eq=b, bounds=bounds, integrality=1, method='highs')

    if res.x is None:
        return 0

    return int(res.x @ [3, 1])

# Part 1

In [78]:
total_tokens = 0
for i in range(problems.shape[0]):
    res = return_optimal_solution(problems[i])
    total_tokens += res

print(total_tokens)

37128


# Part 2

In [79]:
# linprog does not seem to work with such big numbers. back to algebra it is.
# changing the tolerance for np.isclose to 1e-4 was the solution. I guess
# linprog failed because an internal call to np.isclose with a tolerance that was to big  

part_2_offset = 10000000000000
problems[:, 2] += part_2_offset

def return_optimal_solution2(problem: np.ndarray) -> int:
    # A @ x = b ==> x = A^-1 @ b
    A = problem[:2].T
    b = problem[2]
    x = np.linalg.inv(A) @ b
    fractional_part, _ = np.modf(x)
    # check if the result is an integer. With numbers as big as these, an increased tolerance is needed
    if not np.all(np.isclose(fractional_part, 0, atol=1e-4) | np.isclose(fractional_part, 1, atol=1e-4)):
        return 0
    return int(np.rint(x @ [3, 1]))

total_tokens
for i in range(problems.shape[0]):
    total_tokens += return_optimal_solution2(problems[i])
    
print(total_tokens)


74914228508459
